<div style="width: 100%; clear: both;">

<div style="float: left; width: 50%;">

<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">

</div>

<div style="float: right; width: 50%;">

<p style="margin: 0; padding-top: 22px; text-align:right;">M2.991 · Aprenentatge automàtic · PAC2</p>

<p style="margin: 0; text-align:right;">2024-1 · Màster universitari en Ciència de dades (Data science)</p>

<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>

</div>

</div>

<div style="width:100%;">&nbsp;</div>

# PAC 3: Mètodes supervisats

En aquesta pràctica veurem diferents mètodes supervisats i intentarem optimitzar diverses mètriques. Observarem com els diferents models classifiquen les observacions i amb quins obtenim un millor rendiment. Després aplicarem tot el que hem après fins ara a un nou conjunt de dades, simulant un cas pràctic real.

1. [Exploració d’algorismes supervisats](#eje1) \
    1.1. [Càrrega de dades](#eje10) \
    1.2. [Naive-Bayes](#eje11) \
    1.3. [Anàlisi Discriminant Lineal (LDA) i Anàlisi Discriminant Quadràtic (QDA)](#eje12) \
    1.4. [K veïns més propers (KNN)](#eje13) \
    1.5. [Màquines de suport vectorial (SVM)](#eje14) \
    1.6. [Arbres de decisió](#eje15) 
2. [Implementació del cas pràctic](#eje2)\
    2.1. [Càrrega de dades](#eje20) \
    2.2. [Anàlisi Exploratori de Dades](#eje21) \
    2.3. [Preprocessament de Dades](#eje22) \
    2.4. [Modelització](#eje23) 


<u>Consideracions generals</u>:

- La solució plantejada no pot utilitzar mètodes, funcions o paràmetres declarats **_deprecated_** en futures versions, amb l’excepció de la càrrega de dades com s’indica posteriorment.
- Aquesta PAC s’ha de realitzar de forma **estrictament individual**. Qualsevol indici de còpia serà penalitzat amb un suspens (D) per a totes les parts implicades i la possible avaluació negativa de l’assignatura de forma íntegra.
- És necessari que l’estudiant indiqui **totes les fonts** que ha utilitzat per a la realització de la PAC. Si no és així, es considerarà que l’estudiant ha comès plagi, sent penalitzat amb un suspens (D) i la possible avaluació negativa de l’assignatura de forma íntegra.

<u>Format del lliurament</u>:

- Alguns exercicis poden suposar diversos minuts d’execució, per la qual cosa el lliurament s’ha de fer en **format notebook** i en **format HTML**, on es vegi el codi, els resultats i els comentaris de cada exercici. Es pot exportar el notebook a HTML des del menú File $\to$ Download as $\to$ HTML.
- Hi ha un tipus de cel·la especial per albergar text. Aquest tipus de cel·la us serà molt útil per respondre les diferents preguntes teòriques plantejades al llarg de l’activitat. Per canviar el tipus de cel·la a aquest tipus, al menú: Cell $\to$ Cell Type $\to$ Markdown.

<div class="alert alert-block alert-info">
    <strong>Nom i cognoms:</strong>
</div>

In [ ]:
import os
import shap
import copy
import tqdm
import torch
import pickle
import kagglehub
import umap

import seaborn as sns
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim

from matplotlib.colors import ListedColormap

from sklearn import tree
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, train_test_split
from torcheval.metrics.functional import binary_f1_score, binary_accuracy, binary_auroc

%matplotlib inline

<a id='ej1'></a>
# 1. Exploració d’algorismes supervisats

## 1.1. Càrrega de dades

El conjunt de dades Fashion MNIST proporcionat per Zalando consta de 70.000 imatges amb 10 classes diferents de roba repartides uniformement. No obstant això, per a aquesta pràctica utilitzarem únicament un subconjunt de 5.000 imatges que consisteix en 1.000 imatges de 5 classes diferents.

Les imatges tenen una resolució de 28x28 píxels en escala de grisos, per la qual cosa es poden representar utilitzant un vector de 784 posicions.

El següent codi carregarà les 5.000 imatges a la variable `images` i les corresponents etiquetes (en forma numèrica) a la variable `labels`. Podem comprovar que la càrrega ha estat correcta obtenint les dimensions d’aquestes dues variables.

In [ ]:
with open("data.pickle", "rb") as f:
    data = pickle.load(f)
    
X = data["images"]
y = data["labels"]
n_classes = 5
labels = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat"]

print("Vector Image Dimensions: {}".format(X.shape))
print("Vector Label Dimensions: {}".format(y.shape))

Amb el codi següent podem veure un exemple d’imatge de cadascuna de les classes. Per fer-ho, reajustem el vector de 784 dimensions que representa cada imatge en una matriu de mida 28x28 i la transposem per mostrar-la:

In [ ]:
fig, ax = plt.subplots(1, n_classes, figsize=(10,10))

idxs = [np.where(y == i)[0] for i in range(n_classes)]

for i in range(n_classes):
    k = np.random.choice(idxs[i])
    ax[i].imshow(X[k].reshape(28, 28), cmap="gray")
    ax[i].set_title("{}".format(labels[i]))

plt.show()

<div class="alert alert-block alert-info">
    <strong>Implementació:</strong> 

Dividiu el _dataset_ en dos subconjunts, __*train*__ (80% de les dades) i __*test*__ (20% de les dades). Nombreu els conjunts com: X_train, X_test, y_train, y_test. Utilitzeu l’opció `random_state = 24`.
    
Podeu utilitzar la implementació `train_test_split` de `sklearn`.
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

Per tal de visualitzar els resultats de cada algorisme supervisat, reduirem el dataset anterior a dues dimensions.

In [ ]:
model = umap.UMAP(n_components=2, random_state=42)
model.fit(X_train)
X_train_projection = model.transform(X_train)
X_test_projection = model.transform(X_test)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
for i in range(n_classes):
    ax.scatter(X_test_projection[y_test == i,0], X_test_projection[y_test == i,1], s=3, label=labels[i])
plt.legend()
plt.tight_layout()
plt.show()

Al llarg dels exercicis aprendrem a visualitzar gràficament les fronteres de decisió que ens retornen els diferents models. Per fer-ho, utilitzarem la funció definida a continuació, que segueix els passos següents:

1. Crear una meshgrid amb els valors mínim i màxim de 'x' i 'y'.
2. Predir el classificador amb els valors de la meshgrid.
3. Fer un reshape de les dades per obtenir el format corresponent.
4. Un cop fet això, ja podem generar el gràfic de les fronteres de decisió i afegir-hi els punts reals. Així veurem les àrees que el model considera que són d'una classe i les que considera que són d'una altra. Quan hi superposem els punts, podrem comprovar si els classifica correctament dins l'àrea que els correspon.

In [ ]:
# Create the meshgrid with the minimum and maximum values of the x and y axes
x_min, x_max = X_test_projection[:, 0].min() - 1, X_test_projection[:, 0].max() + 1
y_min, y_max = X_test_projection[:, 1].min() - 1, X_test_projection[:, 1].max() + 1

# Define the function that will visualize the decision boundary
def plot_decision_boundaries(model, X_test_projection, y_test):

    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                         np.arange(y_min, y_max, 0.05))

    # Prediction by using all values of the meshgrid
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    # Define the colors (one for each class)
    cmap_light = ListedColormap(['gainsboro','lightgreen','peachpuff','lightcyan', 'pink'])
    cmap_bold = ['grey','g','sandybrown','c','palevioletred']

    # Draw the borders
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(20,10))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Draw the points
    for i in range(n_classes):
         plt.scatter(X_test_projection[y_test == i,0], X_test_projection[y_test == i,1], 
                     s=3, label=labels[i], c=cmap_bold[i])
    plt.legend()
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()

<a id='ej11'></a>
## 1.2. Gaussian Naïve Bayes (1 punt)

El propòsit d’aquest primer exercici és comprendre el funcionament de l’algorisme Naïve-Bayes, un algorisme peculiar que es basa en el teorema de Bayes per calcular la probabilitat que una observació pertanyi a cadascuna de les classes. Aquest model assumeix que les característiques d’entrada són independents entre si, fet que permet simplificar el càlcul de les probabilitats condicionals.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

1. **Entrena un Model de Naïve-Bayes:** Utilitza el conjunt de dades de _train_ per entrenar un model de Naïve-Bayes. Fes servir el classificador `GaussianNB` de la biblioteca `sklearn` per a aquest objectiu.

2. **Calcula l'_Accuracy_ del Model:** Un cop entrenat el model, calcula la seva precisió (_accuracy_) tant en el conjunt de _train_ com en el de _test_. Això et permetrà avaluar com de bé està funcionant el teu model.

3. **Calcula la Matriu de Confusió:** Utilitza el conjunt de _test_ per calcular la matriu de confusió del model. Aquesta matriu t’ajudarà a entendre millor els encerts i errors del teu classificador.

4. **Representa Gràficament la Frontera de Decisió:** Finalment, visualitza la frontera de decisió del model utilitzant el conjunt de _test_. Pots fer això amb l’ajuda de la funció `plot_decision_boundary` que ja has creat prèviament.

Per a realitzar aquests càlculs i visualitzacions, utilitza les funcions `accuracy_score` i `confusion_matrix` del paquet `metrics` de `sklearn`.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong> 
  
Anàlisi de l'exercici.

   - Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algorisme utilitzat?
   - Com són les prediccions obtingudes sobre el conjunt de test?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>


</div>

<a id='ej12'></a>
## 1.3 Anàlisi Discriminant Lineal (LDA) i Anàlisi Discriminant Quadràtic (QDA) (1 punt)

Ara analitzaràs dos algorismes que es basen en la transformació lineal de les característiques d’entrada per maximitzar la separació entre les classes. Aquests models operen sota la suposició que les característiques segueixen una distribució gaussiana. Això et permetrà calcular les probabilitats condicionals de cada classe. Amb aquests càlculs, assignaràs a cada observació la classe que presenti la probabilitat condicional més gran.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Segueix aquests passos amb el dataset d’entrenament (_train_):    
    
1. Entrena un model d’Anàlisi Discriminant Lineal (LDA) utilitzant el classificador `LinearDiscriminantAnalysis` de `sklearn`.
2. Calcula l’_accuracy_ (precisió) del model tant en les dades de _train_ com de _test_.
3. Calcula la matriu de confusió utilitzant les dades de _test_.
4. Representa gràficament la frontera de decisió amb les dades de _test_.

Aquestes accions t’ajudaran a avaluar l’eficàcia del model LDA en el teu conjunt de dades i a entendre millor com classifica les observacions.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. Observa les fronteres de decisió que has generat. Reflexiona sobre la seva forma: s’ajusten al que esperaries de l’algorisme d’Anàlisi Discriminant Lineal (LDA)? Considera la naturalesa lineal de l’algorisme i com això influeix en la forma de les fronteres.
2. Avalua les prediccions realitzades sobre el conjunt de test. Analitza la seva precisió i com es distribueixen respecte a les fronteres de decisió. Són coherents aquestes prediccions amb el que observes a les fronteres de decisió?

Aquestes reflexions et permetran comprendre millor l’efectivitat del model LDA i la seva adequació per al conjunt de dades amb què estàs treballant.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Realitza els passos següents amb el dataset d’entrenament (_train_):

1. Entrena un model d’Anàlisi Discriminant Quadràtic (QDA) utilitzant el classificador `QuadraticDiscriminantAnalysis` de `sklearn`.
2. Calcula l’_accuracy_ (precisió) del model tant en les dades de _train_ com en les de _test_.
3. Calcula la matriu de confusió utilitzant les dades de _test_.
4. Representa gràficament la frontera de decisió amb les dades de _test_.

Aquests passos t’ajudaran a avaluar com el model QDA es comporta amb el teu conjunt de dades i a entendre la seva capacitat per classificar i diferenciar entre les classes.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. Examina les fronteres de decisió que has generat. Reflexiona sobre la seva forma: és coherent amb el que esperaries de l’algorisme d’Anàlisi Discriminant Quadràtic (QDA)? Considera com la naturalesa quadràtica de l’algorisme podria influir en la forma d’aquestes fronteres.
2. Avalua les prediccions realitzades sobre el conjunt de test. Observa la seva precisió i com es distribueixen en relació amb les fronteres de decisió. Són aquestes prediccions consistents amb les fronteres observades?
3. Reflexiona sobre les diferències entre els algorismes LDA i QDA. En què es diferencien en termes de supòsits, enfocament i resultats en les teves dades?

Aquesta anàlisi et permetrà comprendre les característiques i l’eficàcia de tots dos models, LDA i QDA, i com s’apliquen al teu conjunt de dades.

</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<a id='ej13'></a>
## 1.4. KNN (1 punt)

En aquest punt, entendràs el funcionament de l’algorisme KNN (K-Nearest-Neighbor), que es basa en la proximitat dels punts de dades en un espai de característiques. Analitzaràs els seus avantatges i desavantatges, i comprendràs com els paràmetres que el componen influeixen en el seu comportament.

KNN és un algorisme de tipus supervisat basat en instància. Això significa:

- Supervisat: El teu conjunt de dades d’entrenament està etiquetat amb la classe o resultat esperat.
- Basat en instància (_Lazy Learning_): L’algorisme no aprèn explícitament un model, com en la Regressió Logística o els arbres de decisió. En canvi, memoritza les instàncies d’entrenament i les utilitza com a "coneixement" en la fase de predicció.

Per entendre com funciona KNN, segueix aquests passos:

1. Calcula la distància entre l’ítem a classificar i la resta d’ítems del dataset d’entrenament.
2. Selecciona els "k" elements més propers, és a dir, aquells amb la menor distància, segons el tipus de distància que utilitzis (euclidiana, cosinus, manhattan, etc).
3. Realitza una "votació de majoria" entre els k punts seleccionats: la classe que predomini en aquests punts decidirà la classificació final de l’ítem analitzat.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Realitza els passos següents amb el dataset d’entrenament (_train_):

1. Entrena un classificador KNN amb l’hiperparàmetre `n_neighbors=2` utilitzant el classificador `KNeighborsClassifier` de `sklearn`.
2. Calcula l’_accuracy_ (precisió) del model tant en les dades de _train_ com en les de _test_.
3. Calcula la matriu de confusió utilitzant les dades de _test_.
4. Representa gràficament la frontera de decisió amb les dades de _test_.

Si en entrenar el classificador apareix un avís (_warning_) i vols ignorar-lo, executa el codi següent abans de l’entrenament:

`import warnings`  
`warnings.filterwarnings('ignore', message='^.*will change.*$', category=FutureWarning)`

Això et permetrà avaluar l’efectivitat del model KNN amb `n_neighbors=2` en el teu conjunt de dades i entendre com es comporta en termes de classificació i separació de classes.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

En el model que has entrenat, has fixat el paràmetre `n_neighbors` de manera arbitrària. Tanmateix, és possible que amb un altre valor obtinguis una millor predicció. Per trobar el valor òptim dels paràmetres d’un model (_hyperparameter tuning_), sovint s’utilitza una cerca de graella (_grid search_). Això implica entrenar un model per a cada combinació possible d’hiperparàmetres i avaluar-lo mitjançant validació creuada (_cross validation_) amb 5 particions estratificades. Posteriorment, seleccionaràs la combinació d’hiperparàmetres que hagi obtingut els millors resultats.

En aquest cas, et centraràs en optimitzar un sol hiperparàmetre:

- 𝑘: el nombre de veïns que es consideren per classificar un nou exemple. Has de provar amb tots els valors entre 1 i 20.

Realitza aquest procés per identificar el nombre òptim de veïns, fet que et permetrà millorar la precisió de les teves prediccions amb el model KNN.

<div class="alert alert-block alert-info">
    <strong>Implementació:</strong>

Per calcular el valor òptim de l’hiperparàmetre _k_ (`n_neighbors`), has de realitzar una cerca de graella amb validació creuada. Aquest procés t’ajudarà a trobar el valor òptim de _k_. Per a cada valor, calcula la seva mitjana i desviació estàndard. Després, implementa un _heatmap_ per visualitzar la precisió segons els diferents valors de l’hiperparàmetre.

Utilitza el mòdul `GridSearchCV` de `sklearn` per calcular el millor hiperparàmetre. Per a la visualització del _heatmap_, fes servir la biblioteca `Seaborn`.

Aquests passos et permetran identificar de manera efectiva i visual el valor de _k_ que maximitza la precisió del teu model KNN.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Segueix aquests passos amb el dataset d’entrenament (_train_):

1. Entrena un classificador KNN utilitzant el millor hiperparàmetre que hagis trobat.
2. Calcula l’_accuracy_ (precisió) del model tant en les dades de _train_ com en les de _test_.
3. Calcula la matriu de confusió utilitzant les dades de _test_.
4. Representa gràficament la frontera de decisió amb les dades de _test_.

Aquest procés et permetrà veure com l’hiperparàmetre òptim que has identificat millora l’eficàcia del teu model KNN en la classificació de les dades.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. Comenta els resultats obtinguts en la cerca del millor hiperparàmetre. Reflexiona sobre com ha variat el rendiment del model amb els diferents valors de `n_neighbors`.
2. Analitza com es visualitza gràficament el canvi del valor de `n_neighbors`. Observes alguna tendència o patró clar? És coherent aquesta diferència entre els dos gràfics quan canvia el paràmetre?
3. Examina les fronteres de decisió que has generat. Té sentit la forma d’aquestes fronteres donat l’algorisme KNN utilitzat? Pensa en com l’elecció del nombre de veïns influeix en la forma de la frontera.
4. Avalua les prediccions realitzades sobre el conjunt de test. Observa la seva precisió i com es distribueixen en relació amb les fronteres de decisió. Són aquestes prediccions consistents amb el que observes a les fronteres de decisió?

Aquesta anàlisi t’ajudarà a comprendre l’eficàcia del model KNN amb diferents configuracions de `n_neighbors` i el seu impacte en la classificació de les dades.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<a id='ej14'></a>
## 1.5. SVM (1 punt)

En aquesta secció, exploraràs les Màquines de Vectors de Suport (SVM), que es basen en el concepte del _Maximal Margin Classifier_ i l’hiperplà.

Un hiperplà en un espai p-dimensional es defineix com un subespai pla i afí de dimensions p-1. En dues dimensions, és una recta; en tres, un pla convencional. Per a dimensions majors a tres, encara que no sigui intuïtiu de visualitzar, el concepte es manté.

Quan els casos són perfectament separables de manera lineal, sorgeixen infinits possibles hiperplans. Per seleccionar el classificador òptim, utilitza el concepte de _maximal margin hyperplane_, l’hiperplà que es troba més allunyat de totes les observacions d’entrenament. Aquest es defineix calculant la distància perpendicular mínima (marge) de les observacions a un hiperplà. L’hiperplà òptim és aquell que maximitza aquest marge.

En el procés d’optimització, has de tenir en compte que només les observacions al marge o que el violen (vectors suport) influeixen en l’hiperplà. Aquests vectors suport són els que defineixen el classificador.

#### Els _kernels_ en SVM

En situacions on no pots trobar un hiperplà que separi dues classes, és a dir, quan les classes no són linealment separables, pots utilitzar el truc del nucli (_kernel trick_). Aquest mètode et permet treballar en una dimensió nova on és possible trobar un hiperplà per separar les classes.

Igual que amb el KNN, les SVM també depenen de diversos hiperparàmetres. En aquest cas, et centraràs en optimitzar dos hiperparàmetres:

1. **C**: la regularització, que és el valor de penalització dels errors en la classificació. Aquest valor indica el compromís entre obtenir l’hiperplà amb el marge més gran possible i classificar correctament el màxim nombre d’exemples. Has de provar els valors següents: 0.01, 0.1, 1, 10, 50, 100 i 200.
   
2. **Gamma**: un coeficient que multiplica la distància entre dos punts en el kernel radial. En termes simples, com més petit sigui gamma, més influència tindran dos punts propers. Has de provar els valors: 0.001, 0.01, 0.1, 1 i 10.

Per validar el rendiment de l’algorisme amb cada combinació d’hiperparàmetres, utilitza la validació creuada (_cross-validation_) amb 4 particions estratificades.

<div class="alert alert-block alert-info">
    <strong>Implementació:</strong>


1. Calcula el valor òptim dels hiperparàmetres _C_ i _gamma_ utilitzant una cerca de graella amb validació creuada. Aquest procés t’ajudarà a trobar els valors òptims.
2. Per a cada combinació de valors, calcula la seva mitjana i desviació estàndard.
3. Fes un _heatmap_ per visualitzar la precisió segons els diferents valors dels hiperparàmetres.

Utilitza el mòdul `GridSearchCV` de `sklearn` per calcular els millors hiperparàmetres amb el classificador SVC (de `SVM` de `sklearn`). Per a la visualització del _heatmap_, fes servir la biblioteca `Seaborn`.

Aquests passos et permetran identificar de manera efectiva i visual els valors de _C_ i _gamma_ que maximitzen la precisió del teu model SVM.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Realitza els passos següents amb el dataset d’entrenament (_train_):

1. Entrena un model SVM utilitzant la millor combinació de paràmetres que hagis trobat.
2. Calcula l’_accuracy_ (precisió) del model tant en les dades de _train_ com en les de _test_.
3. Calcula la matriu de confusió utilitzant les dades de _test_.
4. Representa gràficament la frontera de decisió amb les dades de _test_.

Aquest procés et permetrà veure com la millor combinació de paràmetres millora l’eficàcia del teu model SVM en la classificació de les dades.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. Comenta els resultats obtinguts en la cerca dels millors hiperparàmetres. Reflexiona sobre com ha variat el rendiment del model SVM amb els diferents valors de _C_ i _gamma_. Considera si els valors òptims trobats tenen sentit en el context del teu conjunt de dades.
2. Examina les fronteres de decisió que has generat amb el model SVM. És coherent la forma d’aquestes fronteres amb el que esperaries de l’algorisme utilitzat? Pensa en com la combinació d’hiperparàmetres seleccionats podria influir en la forma de les fronteres.
3. Avalua les prediccions realitzades sobre el conjunt de test. Observa la seva precisió i com es distribueixen en relació amb les fronteres de decisió. Són aquestes prediccions consistents amb el que observes a les fronteres de decisió?

Aquesta anàlisi t’ajudarà a comprendre l’eficàcia del model SVM amb els hiperparàmetres seleccionats i el seu impacte en la classificació de les dades.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<a id='ej15'></a>
## 1.6. Arbres de decisió (1 punt)

En aquesta secció, exploraràs els arbres de decisió, models predictius que es basen en regles binàries (sí/no) per classificar les observacions segons els seus atributs i predir el valor de la variable resposta. Aquests arbres poden ser classificadors, com en el teu exemple, o regressors per predir variables contínues.

#### Construcció d’un Arbre

Per construir un arbre, segueix l’algorisme de *recursive binary splitting*:

1. Comença a la part superior de l’arbre, on totes les observacions pertanyen a la mateixa regió.
2. Identifica tots els possibles punts de tall per a cadascun dels predictors. Aquests punts de tall són els diferents nivells dels predictors.
3. Avalua les possibles divisions per a cada predictor utilitzant una mesura específica. En els classificadors, aquestes mesures poden ser el *classification error rate*, l’índex Gini, l’entropia o el chi-square.

Comprendre aquests passos t’ajudarà a entendre com els arbres de decisió creen divisions binàries per classificar les dades i com es poden aplicar tant per a classificació com per a regressió.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Sigue estos pasos:

1. Con el dataset de entrenamiento (_train_), entrena un árbol de decisión utilizando el clasificador `DecisionTreeClassifier` de la biblioteca `tree` de `sklearn`.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.
5. Representa el árbol de decisión. Puedes utilizar el comando `plot.tree` de la biblioteca `tree` de `sklearn`.

Estos pasos te permitirán evaluar cómo el árbol de decisión se comporta en tu conjunto de datos, tanto en términos de clasificación como en su representación visual."
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. **Avalua i comenta els resultats:** Analitza l’_accuracy_ del model en els conjunts de _train_ i _test_. Si el model té un _accuracy_ molt alt en _train_ però significativament més baix en _test_, això pot indicar overfitting. A més, revisa la matriu de confusió per identificar si hi ha desequilibris en les classificacions correctes i errors per classe.

2. **Reflexiona sobre la frontera de decisió:** Observa si la frontera de decisió al conjunt de _test_ mostra patrons abruptes o sobrecomplexos. Aquesta característica és coherent amb el que esperaries d’un arbre de decisió, especialment si està profundament ajustat al conjunt de _train_.

3. **Observa la representació gràfica de l’arbre:** Analitza com les ramificacions mostren les decisions preses pel model. Busca si hi ha nodes fulla amb un nombre molt reduït d’observacions o divisions molt específiques que podrien indicar sobreajustament. Comprova si les regles de classificació són intuïtives i si reflecteixen els patrons esperats en el conjunt de dades.

Aquesta anàlisi t’ajudarà a identificar possibles ajustaments al model per millorar la seva capacitat de generalització i la seva eficàcia global.

</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

#### Evitant el *overfitting*

El procés de construcció d’arbres descrit tendeix a reduir ràpidament l’error d’entrenament, de manera que, generalment, el model s’ajusta molt bé a les observacions utilitzades com a entrenament (conjunt de *train*). Com a conseqüència, els arbres de decisió tendeixen a l’*overfitting*.
   
Per evitar l’*overfitting* en els arbres de decisió, és crucial que modifiquis certs hiperparàmetres del model de la manera següent:

1. Utilitza l’hiperparàmetre `max_depth`, que defineix la profunditat màxima de l’arbre. Hauràs d’explorar els valors entre 4 i 10 per trobar l’equilibri adequat entre la complexitat del model i la seva capacitat per generalitzar.
2. Estableix l’hiperparàmetre `min_samples_split`, que és el nombre mínim d’observacions que ha de tenir una fulla de l’arbre abans de considerar una divisió. Experimenta amb valors com 2, 10, 20, 50 i 100 per assegurar-te que l’arbre no es torni massa específic per a les observacions d’entrenament.

Ajustant aquests hiperparàmetres, podràs controlar la tendència de l’arbre de decisió a sobreajustar-se al conjunt d’entrenament, millorant així la seva capacitat per fer prediccions efectives en dades noves.

<div class="alert alert-block alert-info">
    <strong>Implementació:</strong>

1. Calcula el valor òptim dels hiperparàmetres `max_depth` i `min_samples_split` utilitzant una cerca de graella amb validació creuada. Aquest procés t’ajudarà a trobar els valors òptims que evitaran el sobreajustament.
2. Per a cada combinació de valors, calcula la seva mitjana i desviació estàndard.
3. Fes un _heatmap_ per visualitzar la precisió segons els diferents valors dels hiperparàmetres.

Utilitza el mòdul `GridSearchCV` de `sklearn` per calcular els millors hiperparàmetres amb el classificador `DecisionTreeClassifier` de `tree` de `sklearn`. Per a la visualització del _heatmap_, fes servir la biblioteca `Seaborn`.

Aquests passos et permetran identificar de manera efectiva i visual els valors de `max_depth` i `min_samples_split` que maximitzen la precisió del teu arbre de decisió, minimitzant el risc de sobreajustament.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>


1. Entrena un arbre de decisió amb el dataset d’entrenament (_train_) utilitzant la millor combinació de paràmetres que hagis trobat.
2. Calcula l’_accuracy_ (precisió) del model tant en les dades de _train_ com en les de _test_.
3. Calcula la matriu de confusió utilitzant les dades de _test_.
4. Representa gràficament la frontera de decisió amb les dades de _test_.
5. Representa l’arbre de decisió.

Aquests passos et permetran avaluar com l’arbre de decisió, ajustat amb els hiperparàmetres òptims, es comporta en el teu conjunt de dades, tant en termes de classificació com en la seva representació visual.
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. Avalua i comenta els resultats obtinguts en la cerca dels millors hiperparàmetres. Considera com la combinació òptima de `max_depth` i `min_samples_split` ha impactat en el rendiment de l’arbre de decisió.
2. Examina les fronteres de decisió generades amb el conjunt de _test_. Reflexiona si la forma d’aquestes fronteres és coherent amb el que esperaries d’un arbre de decisió configurat amb aquests hiperparàmetres.
3. Analitza les prediccions realitzades sobre el conjunt de _test_. Observa la seva precisió i com es distribueixen en relació amb les fronteres de decisió. Són consistents aquestes prediccions amb l’estructura de l’arbre de decisió i les fronteres observades?

Aquesta anàlisi t’ajudarà a comprendre l’eficàcia de l’arbre de decisió amb els hiperparàmetres seleccionats i el seu impacte en la classificació de les dades.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<a id='eje2'></a>
# 2. Implementació del cas pràctic (5 punts)

Avui dia, la logística de l’última milla és un problema abordat a la indústria per moltes empreses dedicades al comerç electrònic. La informació proporcionada a l’usuari a l’hora de fer una comanda pot suposar un valor diferencial. Per això, moltes empreses dediquen molts recursos per oferir una estimació precisa sobre el temps que trigarà a arribar cada comanda. En aquest exercici ens centrarem a predir el nivell de servei de les operacions logístiques de l’última milla d’Amazon. En concret, identificarem aquelles entregues que es considerin premium (temps de repartiment inferior a dues hores).

Per fer-ho, utilitzarem el conjunt de dades d’entregues [amazon-delivery-dataset](https://www.kaggle.com/datasets/sujalsuthar/amazon-delivery-dataset), el qual inclou dades sobre més de 43.632 entregues en diverses ciutats, amb informació rellevant sobre els detalls de la comanda, els agents de lliurament, les condicions meteorològiques i del trànsit, i les mètriques de rendiment del lliurament. En concret, el dataset conté 16 característiques:

- Order_ID: identificador únic de comanda
- Agent_Age: edat de l’agent (repartidor)
- Agent_Rating: puntuació de l’agent (repartidor)
- Store_Latitude: latitud del magatzem o botiga
- Store_Longitude: longitud del magatzem o botiga
- Drop_Latitude: latitud del client
- Drop_Longitude: longitud del client
- Order_Date: data de la comanda
- Order_Time: hora de la comanda
- Pickup_Time: hora a la qual la comanda va ser recollida per al seu lliurament
- Weather: informació sobre la climatologia
- Traffic: informació sobre el trànsit
- Vehicle: informació sobre el vehicle
- Area: informació sobre l’àrea de repartiment
- Category: categoria dels productes de la comanda
- Delivery_Time: temps de lliurament (minuts)

L’objectiu d’aquesta secció és abordar l’anàlisi d’aquest conjunt de dades i entrenar una xarxa neuronal (Perceptró Multicapa) per predir el nivell de servei. Aquí tens alguns passos que podries seguir:

1. **Anàlisi Exploratori de Dades (EDA)**: Comença explorant el conjunt de dades per entendre’n l’estructura i distribució. Analitza la proporció de cada classe. Observa la distribució de les diferents característiques i la seva relació amb la classe objectiu "class".

2. **Preprocessament de Dades**: Considera normalitzar les característiques perquè estiguin a la mateixa escala que les components principals.

3. **Modelització**: Utilitza un perceptró multicapa com a eina de classificació. Atès que l’objectiu és identificar el nivell de servei del lliurament, és vital centrar-se en mètriques com la precisió, la sensibilitat (_recall_), el valor F1 i l’àrea sota la corba ROC (AUC-ROC

<a id='ej20'></a>
## 2.1. Càrrega de dades i processament inicial (0.5 punts)

El primer que has de fer és carregar el conjunt de dades i visualitzar informació rellevant del mateix. Assegura’t de verificar el següent:

1. Confirma la quantitat total de files i columnes al DataFrame.
2. Revisa el nom de cada columna del DataFrame.
3. Verifica el nombre de valors no nuls en cada columna.
4. Identifica el tipus de dades de cada columna, que pot ser int, float, object, entre d’altres.
5. Comprova la quantitat de memòria utilitzada pel DataFrame.

Aquests passos et proporcionaran una comprensió inicial clara i detallada del conjunt de dades amb el qual estàs treballant.

In [ ]:
root_path = kagglehub.dataset_download("sujalsuthar/amazon-delivery-dataset")
dataset_path = os.path.join(root_path, "amazon_delivery.csv") 
data = pd.read_csv(dataset_path)

data.info()

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Com es pot observar, no disposem del nivell de servei en el conjunt de dades. Per això, definirem que totes les entregues realitzades en un màxim de dues hores han tingut un servei Premium. Per fer-ho, crea una nova columna denominada "Premium_Delivery", que contingui el valor 1 si l’entrega s’ha realitzat en un màxim de 120 minuts, i un 0 en cas contrari. És important assegurar que el tipus de la nova columna sigui enter.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Per simplificar l’exercici i facilitar-ne la comprensió, s’han d’eliminar les següents columnes: Order_ID, Order_Date, Order_Time i Pickup_Time.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>
    
A continuació, calcularem la distància haversiana en quilòmetres entre el magatzem i el client. Per fer-ho, hem de crear una nova columna anomenada "Distance" i eliminar les quatre columnes relacionades amb les coordenades.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<a id='ej21'></a>
## 2.2. Anàlisi Exploratori de Dades (EDA) (1.25 punts)

L’Anàlisi Exploratori de Dades (EDA, per les seves sigles en anglès) en ciència de dades és un enfocament inicial per comprendre i resumir el contingut d’un conjunt de dades. Aquest procés implica diverses tècniques i passos:

1. **Inspecció de Dades**: Es comença revisant les dades brutes per identificar-ne l’estructura, la mida i el tipus (com numèric, categòric). Això inclou detectar valors faltants o inusuals.

2. **Resum Estadístic**: Es calculen estadístiques descriptives com la mitjana, mediana, rang, variància i desviació estàndard per obtenir una idea general de les tendències i patrons en les dades.

3. **Visualització de Dades**: S’utilitzen gràfics i diagrames (com histogrames, gràfics de caixa, diagrames de dispersió) per visualitzar distribucions, relacions entre variables i possibles anomalies. Això ajuda a comprendre millor les dades i a identificar patrons o irregularitats.

4. **Anàlisi de Relacions i Correlacions**: S’exploren les relacions entre diferents variables per entendre com s’influeixen entre si. Això pot implicar l’ús de matrius de correlació i gràfics de dispersió.

5. **Identificació de Patrons i Anomalies**: Es busquen patrons consistents o anomalies (com valors atípics) que puguin suggerir tendències o problemes en les dades.

L’EDA és una fase crítica en qualsevol projecte de ciència de dades, ja que proporciona una comprensió profunda i una base sòlida per a posteriors anàlisis i modelatge.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

1. Calcula les freqüències de la variable objectiu (`Premium_Delivery`) en el teu conjunt de dades. 
2. Crea un gràfic de barres per visualitzar aquestes freqüències. Això t’ajudarà a entendre la proporció d’entregues premium en comparació amb les que no ho són.

A continuació, analitza la distribució de les variables numèriques:

1. Representa gràficament l’histograma de les variables, separant les observacions segons la classe a la qual pertanyen (premium o no).
2. Organitza tots els histogrames en un format de 4 files i 1 columna. Això facilitarà la comparació visual de les distribucions per a cada classe en cada variable.

Per últim, analitza la distribució de les variables categòriques de forma anàloga a les variables numèriques, organitzant tots els histogrames en un format de 5 files i 1 columna.

Aquests passos et permetran obtenir una visió més clara de l’estructura del teu conjunt de dades i com les diferents variables poden influir en la identificació de les entregues premium.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

1. **Avalua la relació de les freqüències de la variable `Premium_Delivery`:** Reflexiona sobre com es distribueixen les transaccions entre premium i no premium. És la distribució significativament desigual? Què implica això per a l’anàlisi i la modelització de les dades?

2. **Analitza la informació proporcionada pels histogrames de les variables descriptives:** Observa si hi ha diferències notables en les distribucions d’aquestes variables entre les classes. Pregunta’t: Hi ha variables que mostrin patrons diferents per al nivell de servei?

3. **Considera altres formes de visualització:** Podrien ser útils altres tipus de gràfics per entendre millor les dades? Per exemple:
   - **Diagrames de caixa (boxplots):** Podrien ser útils per visualitzar la distribució de les variables en ambdues classes i identificar valors atípics.
   - **Mapa de calor de la matriu de correlació:** Pot ajudar a entendre les relacions entre les variables i identificar possibles agrupacions o dependències entre elles.

Aquest anàlisi t’ajudarà a obtenir una comprensió més profunda de la naturalesa de les teves dades i a identificar possibles característiques que podrien ser importants per detectar les entregues premium.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<a id='ej22'></a>
## 2.3. Preprocessament de Dades (1.25 punts)

El preprocessament de dades en ciència de dades és un pas crucial que implica la preparació i transformació de dades brutes en un format adequat per a la seva posterior anàlisi i modelatge. Aquest procés inclou diverses tasques essencials:

1. **Neteja de Dades**: S’eliminen o corregeixen dades errònies, incompletes, inexactes o irrellevants. Això pot incloure tractar amb valors faltants, corregir errors d’entrada i gestionar outliers.

2. **Normalització i Escalat**: Les dades es transformen perquè estiguin en una escala comuna, sense distorsionar diferències en els rangs de valors ni perdre informació. Per exemple, escalat min-max o estandardització.

3. **Codificació de Variables Categòriques**: Les variables categòriques (com gènere o país) es converteixen en formats numèrics perquè puguin ser processades per algorismes d’aprenentatge automàtic, utilitzant tècniques com codificació one-hot o codificació d’etiquetes.

4. **Divisió de Dades**: Les dades es divideixen en conjunts d’entrenament, validació i prova, permetent entrenar models, ajustar hiperparàmetres i avaluar el rendiment del model de manera efectiva.

5. **Gestió de Dades Desbalancejades**: En casos de conjunts de dades desbalancejats, s’apliquen tècniques com sobremostreig o submostreig per assegurar que el model no estigui esbiaixat cap a la classe més freqüent.

6. **Enginyeria de Característiques**: Es creen noves variables (característiques) a partir de les dades existents per millorar la capacitat del model per aprendre patrons i fer prediccions.

El preprocessament és essencial per millorar la qualitat de les dades i fer-les més adequades i efectives per a l’anàlisi i el modelatge en projectes de ciència de dades.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong> elimina els atributs categòrics del conjunt de dades i, en el seu lloc, introdueix la transformació d’aquests atributs a tantes variables binàries com categories tinguin. És important que les noves columnes generades siguin de tipus enter. Recorda que la codificació one-hot converteix les etiquetes categòriques en vectors binaris. En aquests vectors, el valor de 1 s’assigna a la posició corresponent a la classe i el valor de 0 a totes les altres posicions. Això facilita que els models d’aprenentatge automàtic processin i entenguin les etiquetes categòriques.
<hr>
Suggeriment: utilitzeu la funció "get_dummies" de "pandas".

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

A la primera secció de l’exercici vam observar que la columna `Agent_Rating` tenia un nombre reduït de valors nuls. En aquest exercici hem d’imputar els valors nuls pel valor mínim de la columna i verificar que cap columna addicional té valors nuls.
</div>


<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Ara dividirem el conjunt de dades. Per fer-ho, segueix aquests passos:

1. Separa els descriptors de la variable resposta. Assigna els descriptors al conjunt `X` i la variable resposta al conjunt `y`.
2. Elimina del conjunt de descriptors la columna `Delivery_Time`, ja que va ser utilitzada per calcular la nostra variable resposta.
3. Divideix el _dataset_ en dos subconjunts: un per a entrenament (_train_) i un altre per a proves (_test_). Assigna el 80% de les dades al conjunt d’entrenament (`X_train`, `y_train`) i el 20% al conjunt de proves (`X_test`, `y_test`). Utilitza la funció `train_test_split` de la biblioteca `model_selection` de `sklearn`. Assegura’t d’utilitzar `random_state = 24` i fes una divisió estratificada per mantenir la mateixa proporció de classes en ambdós conjunts.

</div>


<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

1. Normalitza els descriptors utilitzant el `StandardScaler` de `sklearn`. Això estandarditzarà les característiques restant la mitjana i dividint per la desviació estàndard.
2. Mostra les dimensions del conjunt de descriptors original, del conjunt d’entrenament i del conjunt de prova. Això et permetrà veure com s’han dividit les dades.

<strong>Nota:</strong> Ajusta el `StandardScaler` únicament amb els descriptors d’entrenament per evitar la fuga d’informació o 'data leakage'. La fuga d’informació ocorre quan s’utilitza informació del conjunt de prova o validació en el procés d’ajust del model. És a dir, si ajustes el model d’escalat amb tot el conjunt de dades, estaries utilitzant informació del conjunt de prova o validació en l’ajust, fet que podria donar la impressió que el model és més precís del que realment és. Per tant, assegura’t d’ajustar el `StandardScaler` només amb les dades d’entrenament i després aplicar-lo als conjunts d’entrenament i prova.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Converteix els conjunts de dades d’entrenament i prova en tensors utilitzant el mètode `tensor` de la biblioteca `PyTorch`.
</div>


<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<a id='ej23'></a>
## 2.4. Modelització (2 punts)

El MLP (Perceptró Multicapa) és, sens dubte, una eina poderosa en el camp de l’aprenentatge automàtic i la intel·ligència artificial. Pot manejar tasques de classificació i regressió, fet que el fa versàtil per a una varietat de problemes. La seva capacitat per modelar relacions no lineals complexes el converteix en una elecció popular quan les dades no segueixen patrons lineals simples.

Aquí hi ha alguns punts clau sobre el MLP:

- **Capes i Neurones**: El MLP consta de múltiples capes de neurones, que inclouen una capa d’entrada, una o més capes ocultes i una capa de sortida. Cada neurona d’una capa està connectada a totes les neurones de la capa següent.

- **Funcions d’Activació**: Per introduir no linealitat en el model, s’utilitzen funcions d’activació a les neurones, com la funció sigmoide, ReLU (Rectified Linear Unit) o tangent hiperbòlica. Aquestes funcions permeten al MLP capturar patrons complexos en les dades.

- **Aprenentatge Supervisat**: L’entrenament del MLP implica ajustar els pesos de les connexions entre neurones per minimitzar la diferència entre les sortides produïdes per la xarxa i les sortides desitjades. Això es fa utilitzant algorismes d’aprenentatge supervisat, com el descens del gradient.

- **Ajust d’Hiperparàmetres**: Igual que altres models d’aprenentatge automàtic, el MLP té hiperparàmetres importants, com el nombre de capes ocultes, el nombre de neurones a cada capa, la funció d’activació i la taxa d’aprenentatge. Sovint, és necessari ajustar aquests hiperparàmetres per obtenir un bon rendiment en una tasca específica.

- **Generalització**: Un dels desafiaments en l’entrenament de MLP és evitar el sobreajustament (overfitting), on el model s’adapta massa a les dades d’entrenament i no generalitza bé a dades noves. La regularització i la validació creuada són tècniques comunes per abordar aquest problema.

En aquest context, el MLP pot ser una excel·lent opció per modelar patrons complexos que indiquin quan una entrega serà premium. No obstant això, és important ajustar i avaluar acuradament el model per garantir que funcioni de manera efectiva en aquesta tasca crítica.

Crear i entrenar un MLP amb diverses capes ocultes amb funció d’activació ReLU és una excel·lent elecció. La funció d’activació ReLU (Rectified Linear Unit) s’utilitza comunament a capes ocultes de xarxes neuronals a causa de la seva capacitat per introduir no linealitat al model, permetent-li aprendre patrons complexos a les dades.

D’altra banda, l’enfocament d’apilar capes lineals utilitzant la classe `Linear` de PyTorch és una forma eficaç i senzilla de construir models de xarxes neuronals.

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, n_epochs, batch_size):
    batch_start = torch.arange(0, len(X_train), batch_size)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.BCELoss()

    best_acc = - np.inf
    best_weights = None

    train_loss_hist = []
    train_acc_hist = []
    val_loss_hist = []
    val_acc_hist = []
 
    for epoch in range(n_epochs):
        epoch_loss = []
        epoch_acc = []
        model.train()
        
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:

                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]

                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)

                # backward pass
                optimizer.zero_grad()
                loss.backward()

                # update weights
                optimizer.step()

                # compute and store metrics
                acc = (y_pred.round() == y_batch).float().mean()
                epoch_loss.append(float(loss))
                epoch_acc.append(float(acc))
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )

        # Evaluating the model at the end of each epoch
        model.eval()
        y_pred = model(X_val)
        ce = float(loss_fn(y_pred, y_val))
        acc = float((y_pred.round() == y_val).float().mean())

        train_loss_hist.append(np.mean(epoch_loss))
        train_acc_hist.append(np.mean(epoch_acc))
        val_loss_hist.append(ce)
        val_acc_hist.append(acc)

        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
        # print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy={acc*100:.1f}%")

    model.load_state_dict(best_weights)
    return best_acc, train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>
    
La modelització del MLP la realitzarem amb la biblioteca `PyTorch`. Per fer-ho:

1. Comença creant el model `BinaryServiceLevel`, per al qual és necessari crear una classe que hereti de `nn.Module`.
2. Al constructor (`__init__`), declara les següents capes:
    - Una capa lineal `nn.Linear` d’entrada amb una mida de sortida de 19, i una funció d’activació ReLU `nn.ReLU`.
    - Una capa lineal `nn.Linear` amb una mida de sortida de 19, i una funció d’activació ReLU `nn.ReLU`.
    - Una capa lineal `nn.Linear` de sortida amb una funció d’activació Sigmoid `nn.Sigmoid`.
3. Després, al mètode `forward` enllaça les diferents capes i les seves respectives funcions d’activació en l’ordre definit en el punt anterior.
4. No oblidis mostrar el nombre de paràmetres utilitzant el mètode `.parameters()` del model.
</div>

In [ ]:
class BinaryServiceLevelBase(nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, x):
        return x

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>
  
1. Ara és el moment d’entrenar i validar el model aplicant validació creuada utilitzant `StratifiedKFold` sobre el conjunt d’entrenament, amb un valor de k = 5 i `shuffle = True`.
2. En cada divisió (split), el model s’ha d’entrenar utilitzant la funció `train_model`. Assegura’t d’entrenar amb les dades d’entrenament i validació de cada divisió, establint el nombre d’èpoques en 15 i la mida del lot en 32.
3. En cada iteració s’han de calcular les següents mètriques:
    - Calcula l’exactitud (_accuracy_) per mesurar la precisió de les prediccions.
    - Calcula el valor F1, que és una mesura que combina precisió i sensibilitat.
    - Calcula l’àrea sota la corba ROC (AUC-ROC) per avaluar el rendiment del model en la classificació binària.
4. Finalment, s’ha de mostrar la mitjana de cadascuna de les mètriques calculades en el punt anterior.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>
    
1. Realitza una anàlisi dels resultats i decideix si consideres que aquest model és acceptable.
2. Avalua quina de les mesures de rendiment utilitzades és la més apropiada.
3. Examina la distribució de les classes i planteja una estratègia, si és necessari, per assegurar la fiabilitat de l’estudi realitzat.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>
  
Ara és el moment d’analitzar la fase d’entrenament. Per fer-ho:
 
1. Divideix el conjunt d’entrenament en dos subconjunts: un per a entrenament (train) i un altre per a validació (val), assignant el 80% de les dades al conjunt d’entrenament.
2. Entrena el model amb la funció `model_train`, guardant tots els paràmetres que retorna.
3. Crea gràfics que mostrin la pèrdua (`loss`) tant en l’entrenament com en la validació al llarg de les èpoques.
4. Finalment, genera gràfics que representin l’exactitud (`accuracy`) en l’entrenament i la validació al llarg de les èpoques.

  
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

Quines conclusions pots obtenir de les gràfiques tant de la pèrdua (`loss`) com de l’exactitud (`accuracy`) en l’entrenament i la validació al llarg de les èpoques?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>
  
És hora d’avaluar el rendiment del model en el conjunt de prova. Per fer-ho:
   
1. Realitza la predicció sobre el conjunt de prova.
2. Calcula les mètriques dels apartats anteriors: exactitud (accuracy), F1 score i corba ROC.
 
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Per analitzar el model entrenat, ens basarem en els [Shapley values](https://en.wikipedia.org/wiki/Shapley_value), que introdueixen l’explicació de models d’aprenentatge automàtic. L’objectiu és explicar la predicció d’un model calculant la contribució de cada característica a la predicció. La descripció tècnica del concepte de SHAP és el càlcul dels valors de Shapley a partir de la teoria de jocs. En poques paraules, els valors de Shapley són un mètode per mostrar l’impacte relatiu de cada característica (o variable) que estem mesurant en el resultat final del model d’aprenentatge automàtic, comparant l’efecte relatiu de les entrades amb la mitjana.

Per calcular els _shap values_:

1. Selecciona una mostra de 10000 registres del conjunt d’entrenament.
2. Inicialitza l’'explainer' `shap.DeepExplainer` amb el model entrenat i la mostra anterior.
3. Selecciona una mostra de 400 registres del conjunt d’entrenament.
4. Calcula els _shap values_ utilitzant la mostra anterior.
5. Defineix i mostra un DataFrame amb tres columnes:
   - Mitjana aritmètica del valor absolut dels valors.
   - Desviació típica del valor absolut dels valors.
   - Nom de l’atribut descriptiu.
6. Mostra la representació gràfica dels valors utilitzant la biblioteca SHAP.
 
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>
    
Relaciona la interpretació dels _shap values_ amb l’anàlisi exploratori de les dades realitzat a l’exercici 2.2.
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>
    
Imagina que calculem els _shap values_ per a cada divisió (split) en l’exercici en què vam entrenar el model utilitzant validació creuada. 
- Els anàlisis dels diferents models haurien de ser similars? Per què o per què no?
- Què indicaria si l’anàlisi de cada model varia molt d’un a l’altre?
- Quins usos addicionals podem donar als _shap values_?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong>
</div>